http://scikit-learn.org/stable/auto_examples/feature_stacker.html

Often it is beneficial to combine several methods to obtain good performance. This example shows how to use FeatureUnion to combine features obtained by PCA and univariate selection.
Combining features using this transformer has the benefit that it allows cross validation and grid searches over the whole process.
The combination used in this example is not particularly helpful on this dataset and is only used to illustrate the usage of FeatureUnion.

petal=꽃잎 // sepal=꽃받침
<img src='iris_petal_sepal.png', width=150, height=100>
<img src='iris-data-analysis-example-in-r-3-638.jpg', width=480>

In [11]:
from sklearn.datasets import load_iris
iris = load_iris()
X, y = iris.data, iris.target

In [35]:
import pandas as pd

In [37]:
df = pd.DataFrame(X)

## PCA

In [12]:
# This dataset is way too high-dimensional. Better do PCA:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca

PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

## SelectKBest

In [15]:
# Maybe some original features where good, too?
from sklearn.feature_selection import SelectKBest
selection = SelectKBest(k=1)
selection

SelectKBest(k=1, score_func=<function f_classif at 0x7fc082cdb230>)

## 위에서 선언한 pca, selection을 동시에 작업할 수 있게 만듦

In [16]:
# Build estimator from PCA and Univariate selection:
from sklearn.pipeline import Pipeline, FeatureUnion
combined_features = FeatureUnion([("pca", pca), ("univ_select", selection)])
combined_features

FeatureUnion(n_jobs=1,
       transformer_list=[('pca', PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('univ_select', SelectKBest(k=1, score_func=<function f_classif at 0x7fc082cdb230>))],
       transformer_weights=None)

## iris 데이터 피팅

In [18]:
# Use combined features to transform dataset:
X_features = combined_features.fit(X, y).transform(X)
X_features[:5]

array([[-2.68420713,  0.32660731,  1.4       ],
       [-2.71539062, -0.16955685,  1.4       ],
       [-2.88981954, -0.13734561,  1.3       ],
       [-2.7464372 , -0.31112432,  1.5       ],
       [-2.72859298,  0.33392456,  1.4       ]])

In [28]:
len(X_features)

150

In [41]:
combined_features.fit(X, y)

FeatureUnion(n_jobs=1,
       transformer_list=[('pca', PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('univ_select', SelectKBest(k=1, score_func=<function f_classif at 0x7fc082cdb230>))],
       transformer_weights=None)

## 서포트 벡터 머신

In [19]:
from sklearn.svm import SVC
svm = SVC(kernel="linear")
svm

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

## pipeline

In [20]:
# Do grid search over k, n_components and C:
pipeline = Pipeline([("features", combined_features), ("svm", svm)])
pipeline

Pipeline(steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('pca', PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('univ_select', SelectKBest(k=1, score_func=<function f_classif at 0x7fc082cdb230>))],
       transformer...,
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

## param_grid 생성(gridsearch에 사용할 매개변수)

In [ ]:
param_grid = dict(features__pca__n_components=[1, 2, 3],
                  features__univ_select__k=[1, 2],
                  svm__C=[0.1, 1, 10])

## GridSearchCV 실시

In [10]:
from sklearn.model_selection import GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid=param_grid, verbose=10)
print grid_search.fit(X, y)

Fitting 3 folds for each of 18 candidates, totalling 54 fits
[CV] features__pca__n_components=1, svm__C=0.1, features__univ_select__k=1 
[CV]  features__pca__n_components=1, svm__C=0.1, features__univ_select__k=1, score=0.960784, total=   0.0s
[CV] features__pca__n_components=1, svm__C=0.1, features__univ_select__k=1 
[CV]  features__pca__n_components=1, svm__C=0.1, features__univ_select__k=1, score=0.901961, total=   0.0s
[CV] features__pca__n_components=1, svm__C=0.1, features__univ_select__k=1 
[CV]  features__pca__n_components=1, svm__C=0.1, features__univ_select__k=1, score=0.979167, total=   0.0s
[CV] features__pca__n_components=1, svm__C=1, features__univ_select__k=1 
[CV]  features__pca__n_components=1, svm__C=1, features__univ_select__k=1, score=0.941176, total=   0.0s
[CV] features__pca__n_components=1, svm__C=1, features__univ_select__k=1 
[CV]  features__pca__n_components=1, svm__C=1, features__univ_select__k=1, score=0.921569, total=   0.0s
[CV] features__pca__n_components

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.2s remaining:    0.0s


[CV] features__pca__n_components=1, svm__C=0.1, features__univ_select__k=2 
[CV]  features__pca__n_components=1, svm__C=0.1, features__univ_select__k=2, score=0.960784, total=   0.0s
[CV] features__pca__n_components=1, svm__C=0.1, features__univ_select__k=2 
[CV]  features__pca__n_components=1, svm__C=0.1, features__univ_select__k=2, score=0.921569, total=   0.0s
[CV] features__pca__n_components=1, svm__C=0.1, features__univ_select__k=2 
[CV]  features__pca__n_components=1, svm__C=0.1, features__univ_select__k=2, score=0.979167, total=   0.0s
[CV] features__pca__n_components=1, svm__C=1, features__univ_select__k=2 
[CV]  features__pca__n_components=1, svm__C=1, features__univ_select__k=2, score=0.960784, total=   0.0s
[CV] features__pca__n_components=1, svm__C=1, features__univ_select__k=2 
[CV]  features__pca__n_components=1, svm__C=1, features__univ_select__k=2, score=0.921569, total=   0.0s
[CV] features__pca__n_components=1, svm__C=1, features__univ_select__k=2 
[CV]  features__pca

[Parallel(n_jobs=1)]: Done  54 out of  54 | elapsed:    1.2s finished


In [6]:
print(grid_search.best_estimator_)

Pipeline(steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('pca', PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('univ_select', SelectKBest(k=2, score_func=<function f_classif at 0x7fc082cdb230>))],
       transformer...,
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])
